# Based
- https://www.kaggle.com/code/umar47/detect-ai-eda-tensorflow-0-91?scriptVersionId=159346087

In [ ]:
import polars as pl
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train = pl.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
test = pl.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')

max_features = 30000
embedding_dim = 64
sequence_length = 512
batch_size = 32
n_splits = 5

In [ ]:
def tf_lower_and_split_punct(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '[^a-z.?!,¿]', '')
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    text = tf.strings.strip(text)
    return text

vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_features,
    ngrams=(3, 5),
    output_mode="int",
    output_sequence_length=sequence_length,
    pad_to_max_tokens=True
)

text_ds = tf.data.Dataset.from_tensor_slices(train['full_text'].to_numpy()).batch(batch_size)
vectorize_layer.adapt(text_ds)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

def create_model():
    inputs = Input(shape=(sequence_length,))
    x = Embedding(max_features, embedding_dim, embeddings_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    x = Dropout(0.5)(x)
    x = Conv1D(64, 5, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = LSTM(64, return_sequences=True)(x)
    x = MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='linear')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=222)

predictions_list = []

for train_index, val_index in kfold.split(train):
    train_df = train[train_index]
    val_df = train[val_index]
    
    raw_train_ds = tf.data.Dataset.from_tensor_slices(
        (train_df['full_text'].to_numpy(), train_df['score'].to_numpy())
    ).batch(batch_size)
    
    raw_val_ds = tf.data.Dataset.from_tensor_slices(
        (val_df['full_text'].to_numpy(), val_df['score'].to_numpy())
    ).batch(batch_size)
    
    train_ds = raw_train_ds.map(vectorize_text)
    val_ds = raw_val_ds.map(vectorize_text)
    
    model = create_model()
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model.fit(train_ds, validation_data=val_ds, epochs=9, callbacks=[early_stopping])
    
    test_texts = test['full_text'].to_numpy()
    test_texts = tf.expand_dims(test_texts, -1)
    test_texts = vectorize_layer(test_texts)
    
    predictions = model.predict(test_texts)
    predictions_list.append(predictions)

In [ ]:
final_predictions = np.mean(predictions_list, axis=0)
final_predictions = np.round(final_predictions).astype(int)
final_predictions = np.clip(final_predictions, 1, 6)

submission = test.select('essay_id').with_columns(score=final_predictions.flatten())
submission.write_csv('submission.csv')

submission.head()

### Please upvote if you find this useful.